In [1]:
from StreetGraph import StreetGraph
from Graphworld import GraphEnv

#graph_meinheim=StreetGraph('meinheim')
#graph_meinheim_trips = graph_meinheim.trips

In [2]:
env=GraphEnv()

In [3]:
env.graph.get_nodeid_by_index(env.position)

290333444

In [2]:
graph_meinheim=StreetGraph('meinheim')
graph_meinheim_trips = StreetGraph('meinheim').trips
print(graph_meinheim_trips)

      tripid  pickup_node  dropoff_node  pickup_day  pickup_hour  \
0          0    307664242     290333858           1           13   
1          1    280324485     306225115           1            6   
2          2    306221807     291556214           1            4   
3          3    306221765     307662889           1           14   
4          4    309441562     354863048           1           19   
...      ...          ...           ...         ...          ...   
3995    3995    306225115     309450366           1           20   
3996    3996    307662889     306219024           1            0   
3997    3997    280324491     290333400           1            3   
3998    3998    309450340     306219024           1            5   
3999    3999    307661801     309450229           1           18   

      pickup_minute     pickup_datetime  \
0                53 2022-01-01 13:53:00   
1                50 2022-01-01 06:50:00   
2                14 2022-01-01 04:14:00   
3          

In [3]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(30):
        print(env.action_space)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("sum_reward: ",sum_reward, " time: ",env.time,  "deadline time: ", env.deadline,"pickup time: ", env.pickup_time )
            break

        print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

env=GraphEnv()
for i in range(1):
    run_one_episode (env)

Discrete(7)
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 11, 7, 7), 'target_node': 307664334, 'route': [290333444, 290333443, 307664257, 307664242, 307664334]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 5, 21), 'target_node': 307664242, 'route': [290333444, 290333443, 307664257, 307664242]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 6, 58), 'target_node': 290333400, 'route': [290333444, 290333400]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 7, 7), 'target_node': 290333400, 'route': [290333444, 290333400]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 4, 36), 'target_node': 307664242, 'route': [290333444, 290333443, 307664257, 307664242]}]
action ==  4  New Position 4
sum_reward:  -1  time:  2022-01-01 11:07:01.600000 deadline time:  2022-01-01 14:04:00 pickup time:  2022-01-01 11:04:00
Discrete(2)
[[{'type': 'wait'}], [{'type': 'ownRide'}]]
action ==  ownRide 
sum_reward:  99  time:  2022-01-01 11:

In [3]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [4]:
ray.init()

2022-04-01 10:50:31,642	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2022-04-01_10-50-29_651840_27695/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-01_10-50-29_651840_27695/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-04-01_10-50-29_651840_27695',
 'metrics_export_port': 61861,
 'gcs_address': '127.0.0.1:61220',
 'node_id': '74b96ae2cb999b58ed7e5277d12e444f8d0cbd6f9b85cc507675f0af'}

In [5]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10
trainer_config["framework"] = "torch"

In [6]:
trainer = PPOTrainer(trainer_config,GraphEnv )

2022-04-01 10:50:39,883	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-04-01 10:50:39,884	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=27777) 2022-04-01 10:50:45,758	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-01 10:50:46,575	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [7]:
checkpoint_root = "tmp/ppo/gridworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [8]:
trainer.train()

(RolloutWorker pid=27777) action ==  2  New Position 2
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27777) action == wait 
(RolloutWorker pid=27

{'episode_reward_max': nan,
 'episode_reward_min': nan,
 'episode_reward_mean': nan,
 'episode_len_mean': nan,
 'episode_media': {},
 'episodes_this_iter': 0,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [], 'episode_lengths': []},
 'sampler_perf': {},
 'off_policy_estimator': {},
 'num_healthy_workers': 1,
 'timesteps_total': 400,
 'timesteps_this_iter': 400,
 'agent_timesteps_total': 400,
 'timers': {'sample_time_ms': 315854.824,
  'sample_throughput': 1.266,
  'load_time_ms': 1.167,
  'load_throughput': 342811.933,
  'learn_time_ms': 168.288,
  'learn_throughput': 2376.878,
  'update_time_ms': 3.972},
 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0,
     'cur_kl_coeff': 0.19999999999999993,
     'cur_lr': 5.0000000000000016e-05,
     'total_loss': 3997.622039794922,
     'policy_loss': 0.08394809067249298,
     'vf_loss': 3997.5381388346354,
     'vf_explain

In [ ]:
result

NameError: name 'result' is not defined

In [ ]:
results = []
episode_data = []
episode_json = []

for n in range(1):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

2022-03-26 16:39:46,012	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


  1: Min/Mean/Max reward:      nan/     nan/     nan, len mean:      nan. Checkpoint saved to tmp/ppo/gridworld/checkpoint_000002/checkpoint-2


In [ ]:
ray.shutdown()